In [51]:
import requests
import base64
import datetime
from urllib.parse import urlencode

# Client_Id and Client_Secret

In [52]:
client_id = 'a91b43299c1640f0aa948cec94a6f6fe'
client_secret = 'dcb87c21bfaa4644b7fff077bb29c129'

In [53]:
class SpotifyAPI(object):
    access_token = None
    access_token_expires = datetime.datetime.now()
    access_token_did_expires = True
    
    client_id = None
    client_secret = None
    token_url = " https://accounts.spotify.com/api/token"
    
    def __init__(self,client_id,client_secret,*args,**kwargs):
        self.client_id = client_id
        self.client_secret = client_secret
        
    def get_client_credentials(self):
        """
        Returns: a Base 64 encoded string
        """
        client_id = self.client_id
        client_secret = self.client_secret
        if client_id == None or client_secret == None:
            raise Exception("You must set client_Id and client_secret")
        client_creds = f"{client_id}:{client_secret}"
        client_creds_b64 = base64.b64encode(client_creds.encode())
        return client_creds_b64.decode()
        
        
    def get_token_header(self):
        client_creds_b64 = self.get_client_credentials()
        
        return {
            "Authorization" : f"Basic {client_creds_b64}"
        }
        
    
    def get_token_data(self):
        return {
            "grant_type": "client_credentials"
        }

    
    def perform_auth(self):
        token_url = self.token_url
        token_data = self.get_token_data()
        token_headers = self.get_token_header()
        req = requests.post(token_url, data=token_data, headers=token_headers)
        if req.status_code not in range(200,299):
             raise Exception("could not Authenticate the client")
        
        data = req.json()
        now = datetime.datetime.now()
        access_token = data['access_token']
        expires_in = data['expires_in'] #seconds
        expires = now + datetime.timedelta(seconds=expires_in)
        self.access_token = access_token
        self.access_token_expires = expires
        self.access_token_did_expires = expires < now
        return True
    
    
    def get_access_token(self):
        token = self.access_token
        expires = self.access_token_expires
        now = datetime.datetime.now()
        if expires<now:
            self.perform_auth()
            return self.get_access_token()
        elif token == None:
            self.perform_auth()
            return self.get_access_token()
     
        return token
    
    def get_resource_header(self):
        access_token = self.get_access_token()
        headers = {
            "Authorization" : f"Bearer {access_token}" 
        }
        return headers
    
    def get_resource(self,lookup_id,resource_type = 'albums',version='v1'):
        endpoint = f" https://api.spotify.com/{version}/{resource_type}/{lookup_id}"
        headers = self.get_resource_header()
        r = requests.get(endpoint,headers=headers)
        if r.status_code not in range(200,299):
            return {}
        return r.json()

        
            
    def get_album(self,_id):
        return self.get_resource(_id,resource_type='albums')

    def get_artists(self,_id):
        return self.get_resource(_id,resource_type='artists')
        
    
    def search(self, query, search_type = "artist"):
        headers = self.get_resource_header()
        endpoint = " https://api.spotify.com/v1/search"
        data = urlencode({"q": query, "type": search_type.lower()})
        look_url = f"{endpoint}?{data}"
        print(look_url)
        r = requests.get(look_url, headers = headers)
        if r.status_code not in range(200, 299):
            return {}
        
        return r.json()
            


        
    

In [54]:
spotify.get_access_token()

'BQA39xFqXIGuxo6_BvHGQm7NBH0E_yTTCktTD11XjKebf9Rhtzy4WdgMywCAoAtb8H5s1IhcCxCGLDnsQtg'

In [55]:
spotify = SpotifyAPI(client_id,client_secret)


In [58]:

spotify.search("Starboy",search_type="Track")

 https://api.spotify.com/v1/search?q=Starboy&type=track


{'tracks': {'href': 'https://api.spotify.com/v1/search?query=Starboy&type=track&offset=0&limit=20',
  'items': [{'album': {'album_type': 'album',
     'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/1Xyo4u8uXC1ZmMpatF05PJ'},
       'href': 'https://api.spotify.com/v1/artists/1Xyo4u8uXC1ZmMpatF05PJ',
       'id': '1Xyo4u8uXC1ZmMpatF05PJ',
       'name': 'The Weeknd',
       'type': 'artist',
       'uri': 'spotify:artist:1Xyo4u8uXC1ZmMpatF05PJ'}],
     'available_markets': ['AD',
      'AE',
      'AR',
      'AT',
      'AU',
      'BE',
      'BG',
      'BH',
      'BO',
      'BR',
      'CA',
      'CH',
      'CL',
      'CO',
      'CR',
      'CY',
      'CZ',
      'DE',
      'DK',
      'DO',
      'DZ',
      'EC',
      'EE',
      'EG',
      'ES',
      'FI',
      'FR',
      'GB',
      'GR',
      'GT',
      'HK',
      'HN',
      'HU',
      'ID',
      'IE',
      'IL',
      'IN',
      'IS',
      'IT',
      'JO',
      'JP',
      'KW

In [59]:
spotify.get_artists(_id="1Xyo4u8uXC1ZmMpatF05PJ")

{'external_urls': {'spotify': 'https://open.spotify.com/artist/1Xyo4u8uXC1ZmMpatF05PJ'},
 'followers': {'href': None, 'total': 20224071},
 'genres': ['canadian contemporary r&b', 'canadian pop', 'pop'],
 'href': 'https://api.spotify.com/v1/artists/1Xyo4u8uXC1ZmMpatF05PJ',
 'id': '1Xyo4u8uXC1ZmMpatF05PJ',
 'images': [{'height': 640,
   'url': 'https://i.scdn.co/image/d9a875c37277c35b94c60c00d2cd256db098505d',
   'width': 640},
  {'height': 320,
   'url': 'https://i.scdn.co/image/91c55730d3f156894d744d560afe441af3f5a228',
   'width': 320},
  {'height': 160,
   'url': 'https://i.scdn.co/image/19393c56208fe6c312384879c11ff3da463f1304',
   'width': 160}],
 'name': 'The Weeknd',
 'popularity': 98,
 'type': 'artist',
 'uri': 'spotify:artist:1Xyo4u8uXC1ZmMpatF05PJ'}

In [60]:
spotify.get_album(_id="2ODvWsOgouMbaA5xf0RkJe")

{'album_type': 'album',
 'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/1Xyo4u8uXC1ZmMpatF05PJ'},
   'href': 'https://api.spotify.com/v1/artists/1Xyo4u8uXC1ZmMpatF05PJ',
   'id': '1Xyo4u8uXC1ZmMpatF05PJ',
   'name': 'The Weeknd',
   'type': 'artist',
   'uri': 'spotify:artist:1Xyo4u8uXC1ZmMpatF05PJ'}],
 'available_markets': ['AD',
  'AE',
  'AR',
  'AT',
  'AU',
  'BE',
  'BG',
  'BH',
  'BO',
  'BR',
  'CA',
  'CH',
  'CL',
  'CO',
  'CR',
  'CY',
  'CZ',
  'DE',
  'DK',
  'DO',
  'DZ',
  'EC',
  'EE',
  'EG',
  'ES',
  'FI',
  'FR',
  'GB',
  'GR',
  'GT',
  'HK',
  'HN',
  'HU',
  'ID',
  'IE',
  'IL',
  'IN',
  'IS',
  'IT',
  'JO',
  'JP',
  'KW',
  'LB',
  'LI',
  'LT',
  'LU',
  'LV',
  'MA',
  'MC',
  'MT',
  'MX',
  'MY',
  'NI',
  'NL',
  'NO',
  'NZ',
  'OM',
  'PA',
  'PE',
  'PH',
  'PL',
  'PS',
  'PT',
  'PY',
  'QA',
  'RO',
  'SA',
  'SE',
  'SG',
  'SK',
  'SV',
  'TH',
  'TN',
  'TR',
  'TW',
  'US',
  'UY',
  'VN',
  'ZA'],
 'copyrights':